# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## run the following command before you run the notebook:
```bash
pip install lxml
```

<!-- # def split_city_name(text):
#     if text == 'New York Islanders':
#         return 'New York'
#     if text == 'New Jersey Devils':
#         return 'New Jersey'
#     if text == 'New York Rangers':
#         return 'New York'
#     if text == 'New York Rangers':
#         return 'New York'
#     if text == 'New York Rangers':
#         return 'New York'
#     else:
#         city = text[:(len(text) - len(split_team_name(text))-1)]
#         return city -->

In [71]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

In [193]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df = pd.read_csv("assets/nhl.csv")
nba_df = pd.read_csv("assets/nba.csv")
mlb_df = pd.read_csv("assets/mlb.csv")
nfl_df = pd.read_csv("assets/nfl.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]


def split_team_name_NHL(item):
    if item == 'Toronto Maple Leafs':
        return 'Maple Leafs'
    elif item == 'Columbus Blue Jackets':
        return 'Blue Jackets'
    elif item == 'Vegas Golden Knights':
        return 'Golden Knights'
    elif item == 'Detroit Red Wings':
        return 'Red Wings'
    elif item == 'New York Islanders':
        return 'RangersIslandersDevils'
    elif item == 'New Jersey Devils':
        return 'RangersIslandersDevils'
    elif item == 'New York Rangers':
        return 'RangersIslandersDevils'
    elif item == 'Anaheim Ducks':
        return 'KingsDucks'
    elif item == 'Los Angeles Kings':
        return 'KingsDucks'
    else:
        return item.split()[-1]


def split_team_name_NBA(item):
    if item == 'New York Knicks':
        # if item.contains('New'):
        return 'KnicksNets'
    elif item == 'Brooklyn Nets':
        return 'KnicksNets'
    elif item == 'Los Angeles Clippers':
        return 'LakersClippers'
    elif item == 'Los Angeles Lakers':
        return 'LakersClippers'
    elif item == 'Portland Trail Blazers':
        return 'Trail Blazers'
    else:
        return item.split()[-1]


def split_team_name_MLB(item):
    if item == 'New York Yankees':
        return 'YankeesMets'
    if item == 'New York Mets':
        return 'YankeesMets'

    elif item == 'Los Angeles Dodgers':
        return 'DodgersAngels'
    elif item == 'Los Angeles Angels':
        return 'DodgersAngels'

    elif item == 'San Francisco Giants':
        return 'GiantsAthletics'
    elif item == 'San Francisco Athletics':
        return 'GiantsAthletics'

    elif item == 'Chicago White Sox':
        return 'CubsWhite Sox'
    elif item == 'Chicago Cubs':
        return 'CubsWhite Sox'

    elif item == 'Boston Red Sox':
        return 'Red Sox'
    elif item == 'Toronto Blue Jays':
        return 'Blue Jays'
    else:
        return item.split()[-1]

    
    
def split_team_name_NFL(item):
    if item == 'New York Jets':
        return 'GiantsJets'
    if item == 'New York Giants':
        return 'GiantsJets'

    elif item == 'Los Angeles Chargers':
        return 'RamsChargers'
    elif item == 'Los Angeles Rams':
        return 'RamsChargers'

    elif item == 'San Francisco 49ers':
        return '49ersRaiders'
    elif item == 'San Francisco Raiders':
        return '49ersRaiders'

    else:
        return item.split()[-1]



## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [325]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
def nhl_correlation(): 
    #* preparing NHL data
    df = nhl_df[nhl_df['year'] == 2018]
    NHL_df_modified = df[['team', 'W', 'L']].reset_index(drop=True)
    NHL_df_modified['team'] = NHL_df_modified['team'].str.replace('*', '', regex=True)
    NHL_df_modified = NHL_df_modified[~NHL_df_modified['team'].str.contains('Division')].reset_index(drop=True)
    NHL_df_modified['Team'] = NHL_df_modified['team'].apply(split_team_name_NHL)
    NHL_df_modified['W/L%'] = NHL_df_modified['W'].astype(float)/(NHL_df_modified['L'].astype(float)+NHL_df_modified['W'].astype(float))
    NHL_df_modified['W/L%'] = NHL_df_modified['W/L%'].astype(float)
    NHL_df_modified.drop('team', axis=1, inplace=True)
    NHL_df_modified = NHL_df_modified[['Team', 'W', 'L', 'W/L%']]

    #* preparing cities data
    pd.options.mode.chained_assignment = None  # !supress useless warnings
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
    Cities_modified = cities[['Metropolitan area','Population (2016 est.)[8]', 'NHL']]
    Cities_modified['NHL'].replace("\[[\w\s]+\]", '', regex=True, inplace=True)
    Cities_modified.rename(columns={'Metropolitan area': 'City','Population (2016 est.)[8]': 'Popul', 'NHL': "Team"}, inplace=True)
    Cities_modified['Popul'] = Cities_modified['Popul'].astype(float)

    #* merging data
    merged = pd.merge(NHL_df_modified, Cities_modified, how='inner', on='Team')
    merged = merged.groupby('City').agg({'Popul': np.mean, 'W/L%': np.mean})

    population_by_region = merged["Popul"]
    win_loss_by_region = merged['W/L%']

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    # 0.012486162921209923
nhl_correlation()

0.012486162921209923

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [324]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df = pd.read_csv("assets/nba.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]


def nba_correlation():

    df = nba_df[nba_df['year'] == 2018]
    NBA_modified = df[['team', 'W/L%']].reset_index(drop=True)
    pd.options.mode.chained_assignment = None  # !supress useless warnings
    NBA_modified['team'].replace("[\*\s][\d\s\(]+\)", '', regex=True, inplace=True)
    NBA_modified['Team'] = NBA_modified['team'].apply(split_team_name_NBA)

    Cities_modified = cities[['Metropolitan area','Population (2016 est.)[8]', 'NBA']]
    Cities_modified['NBA'].replace("\[[\w\s]+\]", '', regex=True, inplace=True)
    Cities_modified.rename(columns={'Metropolitan area': 'City','Population (2016 est.)[8]': 'Popul', 'NBA': "Team"}, inplace=True)
    Cities_modified['Popul'] = Cities_modified['Popul'].astype(float)


    Final_df = Cities_modified.merge(NBA_modified, on="Team")
    Final_df = Final_df[['City', 'Team', 'Popul', 'W/L%']]
    Final_df['W/L%'] = Final_df['W/L%'].astype(float)

    # Final_df['W/L%'][0] = Final_df.iloc[0:2]['W/L%'].mean()
    # Final_df['W/L%'][3] = Final_df.iloc[2:4]['W/L%'].mean()
    # Final_df = Final_df.drop([1, 2], axis=0).reset_index(drop=True)

    Final_df = Final_df.groupby('City').agg({'Popul': np.mean, 'W/L%': np.mean})


    population_by_region = Final_df["Popul"]
    win_loss_by_region = Final_df['W/L%']

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nba_correlation()

-0.1763635064218294

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [323]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df = pd.read_csv("assets/mlb.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]


def mlb_correlation():
    pd.options.mode.chained_assignment = None  # !supress useless warnings
    df = mlb_df[mlb_df['year'] == 2018]
    MLB_modified = df[['team', 'W-L%']].reset_index(drop=True)
    MLB_modified['team'].replace("[\*\s][\d\s\(]+\)", '', regex=True, inplace=True)
    MLB_modified['Team'] = MLB_modified['team'].apply(split_team_name_MLB)
    MLB_modified = MLB_modified.drop("team", axis=1)[['Team', 'W-L%']]


    Cities_modified = cities[['Metropolitan area','Population (2016 est.)[8]', 'MLB']]
    Cities_modified['MLB'].replace("\[[\w\s]+\]", '', regex=True, inplace=True)
    Cities_modified.rename(columns={'Metropolitan area': 'City','Population (2016 est.)[8]': 'Popul', 'MLB': "Team"}, inplace=True)


    MLB_modified['W-L%'] = MLB_modified['W-L%'].astype(float)
    Cities_modified['Popul'] = Cities_modified['Popul'].astype(float)

    merged=pd.merge(MLB_modified,Cities_modified,'inner', on = 'Team')
    merged = merged.groupby('City').agg({'W-L%': np.nanmean, 'Popul': np.nanmean})
    merged.loc['San Francisco Bay Area']['W-L%'] = 0.525 #* fixing the San Francisco Bay Area issue

    population_by_region = merged["Popul"]
    win_loss_by_region = merged['W-L%']

    assert len(population_by_region) == len(
        win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(
        population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    # return stats.pearsonr(population_by_region, win_loss_by_region)
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()


0.15003737475409498

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [322]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
pd.options.mode.chained_assignment = None  # !supress useless warnings
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    df = nfl_df[nfl_df['year'] == 2018]
    NFL_modified = df[['team', 'W-L%']].reset_index(drop=True)
    NFL_modified = NFL_modified[~NFL_modified['W-L%'].str.contains('t')].reset_index(drop=True)
    NFL_modified['team'].replace("[\*\+]", '', regex=True, inplace=True)
    NFL_modified['Team'] = NFL_modified['team'].apply(split_team_name_NFL)

    Cities_modified = cities[['Metropolitan area','Population (2016 est.)[8]', 'NFL']]
    Cities_modified['NFL'].replace("\[[\w\s]+\]", '', regex=True, inplace=True)
    Cities_modified.rename(columns={'Metropolitan area': 'City','Population (2016 est.)[8]': 'Popul', 'NFL': "Team"}, inplace=True)

    # Cities_modified
    NFL_modified['W-L%'] = NFL_modified['W-L%'].astype(float)
    Cities_modified['Popul'] = Cities_modified['Popul'].astype(float)

    merged=pd.merge(NFL_modified,Cities_modified,'inner', on = 'Team')
    merged = merged.groupby('City').agg({'W-L%': np.nanmean, 'Popul': np.nanmean})
    merged['W-L%'] = merged['W-L%'].astype(float)
    merged['Popul'] = merged['Popul'].astype(float)
    population_by_region = merged["Popul"]
    win_loss_by_region = merged['W-L%']

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]



nfl_correlation()

0.004282141436393022

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.